In [1]:
from transformers import RobertaModel,RobertaTokenizer
from utils import *

In [2]:
device = torch.device(7)


In [26]:
roberta_model = RobertaModel.from_pretrained('roberta-base').to(device)
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [3]:
import sys
sys.path.append('../..')
from kge.model import KgeModel
from kge.util.io import load_checkpoint


In [4]:
kg_type = 'half'
hops = 'webqsp_half'
checkpoint_file = '../../pretrained_models/embeddings/ComplEx_fbwq_' + kg_type + '/checkpoint_best.pt'
kge_checkpoint = load_checkpoint(checkpoint_file)
kge_model = KgeModel.create_from(kge_checkpoint)
kge_model.eval()
e = getEntityEmbeddings(kge_model, hops)

Loading configuration of dataset fbwq_half...
Setting complex.relation_embedder.dropout to 0, was set to -0.4746062345802784.
Loading half entity_ids.del


In [5]:
from torch.utils.data import Dataset, DataLoader
from dataloader import DatasetMetaQA, DataLoaderMetaQA


In [7]:
data_path = '../../data/QA_data/WebQuestionsSP/qa_train_webqsp.txt'
entity2idx, idx2entity, embedding_matrix = prepare_embeddings(e)
data = process_text_file(data_path, split=False)
print('Train file processed, making dataloader')
# word2ix,idx2word, max_len = get_vocab(data)
# hops = str(num_hops)
device = torch.device(device)
dataset = DatasetMetaQA(data, e, entity2idx)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

Train file processed, making dataloader


In [8]:
a = next(iter(data_loader))

In [24]:
question_tokenized = a[0].to(device)
attention_mask = a[1].to(device)
positive_head = a[2].to(device)
positive_tail = a[3].to(device)  

In [19]:
roberta_tokenizer.decode(question_tokenized[0].tolist(),skip_special_tokens=True)

'what is the name of justin bieber brother NE'

In [14]:
question_tokenized[0]

tensor([    0, 12196,    16,     5,   766,     9,    95,   179,   741,   324,
         1943,  2138, 12462,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1])

In [17]:
test = roberta_tokenizer.encode('There is a test')
roberta_tokenizer.decode(test)

'There is a test'

In [21]:
device


device(type='cuda', index=2)

In [25]:
question_tokenized

tensor([[    0, 12196,    16,     5,   766,     9,    95,   179,   741,   324,
          1943,  2138, 12462,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1]], device='cuda:7')

In [27]:
roberta_model(question_tokenized, attention_mask=attention_mask)

RuntimeError: cublas runtime error : the GPU program failed to execute at /opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCBlas.cu:368